# 人気度順推薦
## 人気度の定義
* 今回は評価値が高いものを人気が髙い映画とする
* 人気度の定義は「クリック数が多いもの」「購入が多いもの」「評価値が髙いもの」など複数あり、自社サービスに最も適した定義を利用


In [1]:
# 親のフォルダのパスを追加
import sys; sys.path.insert(0, '..')

from util.data_loader import DataLoader
from util.metric_calculator import MetricCalculator

In [2]:
# Movielensのデータの読み込み
data_loader = DataLoader(num_users=1000, num_test_items=5, data_path='../data/ml-10M100K/')
movielens = data_loader.load()

In [11]:
import numpy as np
# 評価値が髙い映画の確認
movie_stats = movielens.train.groupby(['movie_id', 'title']).agg({'rating': ['size', 'mean']})
movie_stats.sort_values(by=('rating', 'mean'), ascending=False).head()

rating     
                                            size mean
movie_id title                                       
7831     Another Thin Man (1939)               1  5.0
4354     Unlawful Entry (1992)                 1  5.0
7227     Trouble with Angels, The (1966)       1  5.0
6426     Far Country, The (1954)               1  5.0
32460    Knockin' On Heaven's Door (1997)      1  5.0

In [12]:
# しきい値を導入
movie_stats = movielens.train.groupby(['movie_id', 'title']).agg({'rating': ['size', 'mean']})
atleast_flg = movie_stats['rating']['size'] >= 100
movies_sorted_by_rating = movie_stats[atleast_flg].sort_values(by=('rating', 'mean'), ascending=False)
movies_sorted_by_rating.head()

rating          
                                                       size      mean
movie_id title                                                       
318      Shawshank Redemption, The (1994)               423  4.492908
50       Usual Suspects, The (1995)                     332  4.459337
912      Casablanca (1942)                              163  4.444785
904      Rear Window (1954)                             129  4.441860
2019     Seven Samurai (Shichinin no samurai) (1954)    104  4.408654

In [15]:
# 人気度推薦
from src.popularity import PopularityRecommender
recommender = PopularityRecommender()
recommend_result = recommender.recommend(movielens, minimum_num_rating=100)

/app/notebook/../src/popularity.py:15: FutureWarning: The provided callable <function mean at 0x7f1e59f2bd00> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movie_rating_average = dataset.train.groupby("movie_id").agg({"rating": np.mean})
/app/notebook/../src/popularity.py:25: FutureWarning: The provided callable <function mean at 0x7f1e59f2bd00> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movie_stats = dataset.train.groupby("movie_id").agg({"rating": [np.size, np.mean]})


In [16]:
#  評価
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

rmse=1.089, Precision@K=0.008, Recall@K=0.025


In [17]:
# しきい値を変更したときの挙動
for minimum_num_rating in [1, 200]:
    recommend_result = recommender.recommend(movielens, minimum_num_rating=minimum_num_rating)
    metrics = metric_calculator.calc(
        movielens.test.rating.tolist(), recommend_result.rating.tolist(),
        movielens.test_user2items, recommend_result.user2items, k=10)
    print(metrics)

/app/notebook/../src/popularity.py:15: FutureWarning: The provided callable <function mean at 0x7f1e59f2bd00> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movie_rating_average = dataset.train.groupby("movie_id").agg({"rating": np.mean})
/app/notebook/../src/popularity.py:25: FutureWarning: The provided callable <function mean at 0x7f1e59f2bd00> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movie_stats = dataset.train.groupby("movie_id").agg({"rating": [np.size, np.mean]})
/app/notebook/../src/popularity.py:15: FutureWarning: The provided callable <function mean at 0x7f1e59f2bd00> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string 

rmse=1.089, Precision@K=0.000, Recall@K=0.000
rmse=1.089, Precision@K=0.013, Recall@K=0.040


/app/notebook/../src/popularity.py:25: FutureWarning: The provided callable <function mean at 0x7f1e59f2bd00> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  movie_stats = dataset.train.groupby("movie_id").agg({"rating": [np.size, np.mean]})
